In [1]:
from io import StringIO

from ase.io import read

import openmm
import openmm.app
import openmm.unit

In [2]:
offxml = """
<ForceField>
<AtomTypes>
 <Type name="0" class="Si" element="Si" mass="28.0855"/>
 <Type name="1" class="O" element="O" mass="15.9994"/>
</AtomTypes>

<Residues>
 <Residue name="Si">
  <Atom name="Si" type="0"/>
 </Residue>
 <Residue name="O">
  <Atom name="O" type="1"/>
 </Residue>
</Residues>

<CustomNonbondedForce energy="acoef*exp(-bcoef*r) - ccoef/r^6 + dcoef/r^24;
  acoef = (18003.757 * pairSiO + 1388.7730 * pairOO) * 96.4916;
  bcoef = (4.87318 * pairSiO + 2.76000 * pairOO) * 10.;
  ccoef = (133.5381 * pairSiO + 175.0000 * pairOO) * 96.4916 * 1.e-6;
  dcoef = (29.0 * pairSiO + 113.0 * pairOO + 3423200. * pairSiSi) * 96.4916 * 1.e-24;
  pairOO = IsO1*IsO2; pairSiSi = (1-IsSi1)*(1-IsSi2); pairSiO = (1-IsSi1)*IsSi2 + (1-IsSi2)*IsSi1;
  " bondCutoff="3">
 <PerParticleParameter name="IsO"/>
 <PerParticleParameter name="IsSi"/>
 <Atom type="0" IsO="0" IsSi="1"/>
 <Atom type="1" IsO="1" IsSi="0"/>
</CustomNonbondedForce>

<NonbondedForce coulomb14scale="0." lj14scale="0.">
 <Atom type="0" charge="2.4" sigma="0.35" epsilon="0."/>
 <Atom type="1" charge="-1.2" sigma="0.35" epsilon="0."/>
</NonbondedForce>
</ForceField>
"""

forcefield = openmm.app.forcefield.ForceField(StringIO(offxml))

In [3]:
fau = read("FAU.cif")

topology = openmm.app.Topology()
chain = topology.addChain('SiO2')
angstrom_to_nm = 0.1
topology.setPeriodicBoxVectors([fau.cell.array[0] * angstrom_to_nm,
                                fau.cell.array[1] * angstrom_to_nm, 
                                fau.cell.array[2] * angstrom_to_nm])
for atom in fau:
    res = topology.addResidue('MOL', chain)
    topology.addAtom(atom.symbol, openmm.app.element.Element.getByAtomicNumber(atom.number), res)

system = forcefield.createSystem(topology, nonbondedMethod=openmm.app.CutoffPeriodic, nonbondedCutoff=11. * openmm.unit.angstrom)

/Users/fx/miniforge3/envs/ML2025b/lib/python3.12/site-packages/ase/io/cif.py:410: UserWarning: crystal system 'cubic' is not interpreted for space group Spacegroup(227, setting=1). This may result in wrong setting!
  warnings.warn(


In [4]:
press = 0. * openmm.unit.bar
temp = 200. * openmm.unit.kelvin
friction = 1. / openmm.unit.picoseconds
step = 0.002 * openmm.unit.picoseconds

integrator = openmm.LangevinMiddleIntegrator(temp, friction, step)
simulation = openmm.app.Simulation(topology, system, integrator)

pdb_reporter = openmm.app.PDBReporter("md_trajectory.pdb", 1, enforcePeriodicBox=False)
dcd_reporter = openmm.app.DCDReporter("md_trajectory.dcd", 1, enforcePeriodicBox=False)

state_data_reporter = openmm.app.StateDataReporter("md_data.csv", 1,
    step=True, time=True, potentialEnergy=True, kineticEnergy=True,
    volume=True, temperature=True, density=True
)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(state_data_reporter)

In [5]:
simulation.context.setPositions(fau.positions * openmm.unit.angstrom)
simulation.step(2000)

ValueError: Energy is infinite.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#nan